書籍[pythonで作る対話システム]を基に言語モデルの学習を行うソースコード

In [2]:
!python --version

Python 3.10.12


In [ ]:
!pip check

ipython 7.34.0 requires jedi, which is not installed.
pygobject 3.42.1 requires pycairo, which is not installed.


In [ ]:
!pip install --upgrade pip
!pip install setuptools wheel
!pip3 install torch torchvision torchaudio
!pip install spacy panel==0.6.4 prophet holoviews==1.12.4 configargparse
# !pip install OpenNMT-py
!git clone https://github.com/OpenNMT/OpenNMT-py.git -b 1.0.0
!pip install -U OpenNMT-py


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.8/479.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 7.3 MB/s eta 0:00:00
  Obtaining dependency information for configargparse from https://files.pythonhosted.org/packages/0b/85/798783fa1f6dcb09f87f6ba519a46a86d69d0716b685065a733e850e715f/ConfigArgParse-1.5.5-py3-none-any.whl.metadata
  Attempting uninstall: holoviews
    Found existing installation: holoviews 1.15.4
    Uninstalling holoviews-1.15.4:
      Successfully uninstalled holoviews-1.15.4
  Attempting uninstall: panel
    Found existing installation: panel 0.14.4
    Uninstalling panel-0.14.4:
      Successfully uninstalled panel-0.14.4
Cloning into 'OpenNMT-py'...
remote: Enumerating objects: 20805, done.
remote: Counting 

データの読み込みと保存のためGoogle Driveに接続します．

「このノートブックに Google ドライブのファイルへのアクセスを許可しますか？」と表示されるので「Google Driveに接続」をクリックし，使用するアカウントを選択します．

次のページで「許可」ボタンをクリックしてください．


In [2]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [ ]:
!pip install torchtext==0.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.8 MB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.15.2
    Uninstalling torchtext-0.15.2:
      Successfully uninstalled torchtext-0.15.2


学習を実行します．

In [ ]:
!ls  "drive/My Drive/dsbook"

 config_chat_bot.yaml  'learn_generative_model.ipynb のコピー'	 OpenNMT-py
 example_based_bert     OpenNMT


語彙辞書の作成

In [ ]:
!onmt_build_vocab --config "drive/My Drive/dsbook/config_chat_bot.yaml" --n_sample -1

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2023-07-17 03:10:22,709 INFO] Counter vocab from -1 samples.
[2023-07-17 03:10:22,709 INFO] n_sample=-1: Build vocab on full datasets.
[2023-07-17 03:10:23,159 INFO] Counters src: 7526
[2023-07-17 03:10:23,160 INFO] Counters tgt: 7777
[2023-07-17 03:10:23,165 INFO] Counters after share:7791
[2023-07-17 03:10:23,165 WARNING] path drive/My Drive/dsbook/OpenNMT/vocab.txt exists, may overwrite...


In [ ]:
!head "drive/My Drive/dsbook/OpenNMT/vocab.txt" -n 20

。	17328
です	13801
は	8272
、	6780
が	6736
て	6715
ね	6394
の	6118
た	6052
か	5726
ます	5427
に	4665
？	3872
も	3675
ん	3411
で	3252
と	3237
な	3066
し	2922
！	2636


RNN(LSTM)の学習1

In [ ]:
!onmt_train --config "drive/My Drive/dsbook/test.yaml"

[2023-07-17 04:38:25,871 INFO] Missing transforms field for corpus_1 data, set to default: [].
[2023-07-17 04:38:25,872 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2023-07-17 04:38:25,872 INFO] Missing transforms field for valid data, set to default: [].
[2023-07-17 04:38:25,873 INFO] Parsed 2 corpora from -data.
[2023-07-17 04:38:25,873 INFO] Get special vocabs from Transforms: {'src': [], 'tgt': []}.
[2023-07-17 04:38:25,897 INFO] The first 10 tokens of the vocabs are:['<unk>', '<blank>', '<s>', '</s>', '。', 'です', 'は', '、', 'が', 'て']
[2023-07-17 04:38:25,897 INFO] The decoder start token is: <s>
[2023-07-17 04:38:25,897 INFO] Building model...
[2023-07-17 04:38:26,193 INFO] Switching model to float32 for amp/apex_amp
[2023-07-17 04:38:26,193 INFO] Non quantized layer compute is fp32
[2023-07-17 04:38:28,990 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
   

学習結果の確認

In [ ]:
!onmt_translate -model "drive/My Drive/dsbook/OpenNMT/dlg_model_step_1500.pt"  --src "drive/My Drive/dsbook/OpenNMT/test.src" --output "drive/My Drive/dsbook/OpenNMT/pred_1500_by_lstm1.txt"

[2023-07-17 11:28:11,207 INFO] Loading checkpoint from drive/My Drive/dsbook/OpenNMT/dlg_model_step_1500.pt
[2023-07-17 11:28:12,943 INFO] Loading data into the model
[2023-07-17 11:29:57,524 INFO] PRED SCORE: -0.7842, PRED PPL: 2.19 NB SENTENCES: 2000


RNN(LSTM)の学習2

In [ ]:
!python OpenNMT-py/preprocess.py -train_src "drive/My Drive/dsbook/OpenNMT/train.src" -train_tgt "drive/My Drive/dsbook/OpenNMT/train.tgt" \
 -valid_src "drive/My Drive/dsbook/OpenNMT/dev.src" -valid_tgt "drive/My Drive/dsbook/OpenNMT/dev.tgt" -save_data dlg
!python OpenNMT-py/train.py -gpu_ranks 0 --save_checkpoint_steps 500 --train_steps 3000 -save_model "drive/My Drive/dsbook/OpenNMT/lstm_model2" -data dlg

[2023-07-17 05:47:03,849 INFO] Extracting features...
[2023-07-17 05:47:03,852 INFO]  * number of source features: 0.
[2023-07-17 05:47:03,852 INFO]  * number of target features: 0.
[2023-07-17 05:47:03,852 INFO] Building `Fields` object...
[2023-07-17 05:47:03,853 INFO] Building & saving training data...
[2023-07-17 05:47:03,853 WARNING] Shards for corpus train already exist, won't be overwritten, pass the `-overwrite` option if you want to.
[2023-07-17 05:47:03,854 INFO] Building & saving validation data...
[2023-07-17 05:47:03,854 WARNING] Shards for corpus valid already exist, won't be overwritten, pass the `-overwrite` option if you want to.
[2023-07-17 05:47:07,446 INFO]  * src vocab size = 7487
[2023-07-17 05:47:07,446 INFO]  * tgt vocab size = 7736
[2023-07-17 05:47:07,446 INFO] Building model...
[2023-07-17 05:47:07,992 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embe

学習結果の表示。

In [ ]:
!python OpenNMT-py/translate.py -model "drive/My Drive/dsbook/OpenNMT/lstm_model2_step_1500.pt" -src "drive/My Drive/dsbook/OpenNMT/test.src"  -output "drive/My Drive/dsbook/OpenNMT/pred_1500_by_lstm2.txt"  -replace_unk -verbose

[2023-07-17 11:13:42,441 INFO] Translating shard 0.
/content/OpenNMT-py/onmt/translate/beam_search.py:187: UserWarning: An output with one or more elements was resized since it had shape [150], which does not match the required output shape [30, 5]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at ../aten/src/ATen/native/Resize.cpp:26.)
  torch.mul(self.topk_scores, length_penalty, out=self.topk_log_probs)
Traceback (most recent call last):
  File "/content/OpenNMT-py/translate.py", line 6, in <module>
    main()
  File "/content/OpenNMT-py/onmt/bin/translate.py", line 48, in main
    translate(opt)
  File "/content/OpenNMT-py/onmt/bin/translate.py", line 25, in translate
    translator.translate(
  File "/content/OpenNMT-py/onmt/translate/translator.py", line 351, in translate
    batch_da

transformerモデルの学習

In [ ]:
!onmt_train --config "drive/My Drive/dsbook/config_chat_bot_by_transformer.yaml"

ストリーミング出力は最後の 5000 行に切り捨てられました。
			* corpus_1: 11659
[2023-07-19 05:11:38,418 INFO] Weighted corpora loaded so far:
			* corpus_1: 11660
[2023-07-19 05:11:38,669 INFO] Weighted corpora loaded so far:
			* corpus_1: 11661
[2023-07-19 05:11:38,867 INFO] Weighted corpora loaded so far:
			* corpus_1: 11662
[2023-07-19 05:11:39,121 INFO] Weighted corpora loaded so far:
			* corpus_1: 11663
[2023-07-19 05:11:39,385 INFO] Weighted corpora loaded so far:
			* corpus_1: 11664
[2023-07-19 05:11:39,674 INFO] Weighted corpora loaded so far:
			* corpus_1: 11665
[2023-07-19 05:11:39,887 INFO] Weighted corpora loaded so far:
			* corpus_1: 11666
[2023-07-19 05:11:40,155 INFO] Weighted corpora loaded so far:
			* corpus_1: 11667
[2023-07-19 05:11:40,421 INFO] Weighted corpora loaded so far:
			* corpus_1: 11668
[2023-07-19 05:11:40,636 INFO] Weighted corpora loaded so far:
			* corpus_1: 11669
[2023-07-19 05:11:40,974 INFO] Weighted corpora loaded so far:
			* corpus_1: 11670
[2023-07-19 05:11:41,50

In [ ]:
!onmt_translate -model "drive/My Drive/dsbook/OpenNMT/tsf_model_step_2000.pt"  --src "drive/My Drive/dsbook/OpenNMT/test.src" --output "drive/My Drive/dsbook/OpenNMT/pred_2000_by_tsf.txt"

[2023-07-19 05:45:50,834 INFO] Loading checkpoint from drive/My Drive/dsbook/OpenNMT/tsf_model_step_2000.pt
[2023-07-19 05:46:20,273 INFO] Loading data into the model
[2023-07-19 05:52:11,365 INFO] PRED SCORE: -0.4439, PRED PPL: 1.56 NB SENTENCES: 2000


rinna gpt2(apanese-gpt2-medium)を利用したファインチューニング
学習用データとしてmanga109のデータ使用

パッケージのインストール

In [1]:
# パッケージのインストール
!git clone https://github.com/huggingface/transformers -b v4.23.1
!pip install transformers==4.23.1
!pip install evaluate==0.3.0
!pip install sentencepiece==0.1.97

Cloning into 'transformers'...
remote: Enumerating objects: 157408, done.
remote: Counting objects: 100% (2433/2433), done.
remote: Compressing objects: 100% (847/847), done.
remote: Total 157408 (delta 1501), reused 2025 (delta 1350), pack-reused 154975
Receiving objects: 100% (157408/157408), 159.43 MiB | 17.46 MiB/s, done.
Resolving deltas: 100% (117715/117715), done.
Note: switching to 'bd469c40659ce76c81f69c7726759d249b4aef49'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [7]:
!pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
aiohttp                          3.8.5
aiosignal                        1.3.1
alabaster                        0.7.13
albumentations                   1.3.1
altair                           4.2.2
annotated-types                  0.5.0
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array-record                     0.4.1
arviz                            0.15.1
astropy                          5.3.2
astunparse                       1.6.3
async-timeout                    4.0.3
attrs                            23.1.0
audioread                        3.0.0
autograd                         1.6.2
Babel                            2.12.1
backcall                         0.2.0
beautifulsoup4                   4.11.2
bleach                           6.0.0


rinna gpt2(apanese-gpt2-medium)のファインチューニング

In [5]:
!python ./transformers/examples/pytorch/language-modeling/run_clm.py     --model_name_or_path=rinna/japanese-gpt2-small     --train_file="dialogue_pairs_re.txt"     --do_train     --do_eval     --num_train_epochs=10     --save_steps=10000     --block_size 512     --save_total_limit=3     --output_dir "./output"     --overwrite_output_dir     --use_fast_tokenizer=False

2023-08-22 14:30:32.980915: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-22 14:30:34.323487: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
08/22/2023 14:30:38 - WARNING - __main__ - Process rank: -1, device: cpu, n_gpu: 0distributed training: False, 16-bits training: False
08/22/2023 14:30:38 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=0,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_b

In [6]:
!python ./transformers/examples/pytorch/language-modeling/run_clm.py \
    --model_name_or_path=rinna/japanese-gpt2-small \
    --train_file="drive/My Drive/dsbook/OpenNMT/siraisi_dialogue_re.txt" \
    --do_train \
    --do_eval \
    --num_train_epochs=10 \
    --save_steps=10000 \
    --block_size 512 \
    --save_total_limit=3 \
    --output_dir "drive/My Drive/output_by_siraisi/" \
    --overwrite_output_dir \
    --use_fast_tokenizer=False

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
Moving 0 files to the new cache system
0it [00:00, ?it/s]
2023-08-22 14:34:02.920793: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-22 14:34:05.647815: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
08/22/2023 14:34:12 - WARNING - __main__ - Process rank: -1, device: cpu, n_gpu: 0distributed training: False, 16-bits training: False
08/22/2023 14:34:12 - INFO - __main__ - Traini

学習用データを変えてファインチューニングを実験

In [ ]:
from transformers import T5Tokenizer, AutoModelForCausalLM

# トークナイザーとモデルの準備
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-small")
# 訳7万件のテキストデータでファインチューニングしたもの
model = AutoModelForCausalLM.from_pretrained("drive/My Drive/output")
# 訳3万件の対話テキストデータでファインチューニングしたもの
# model = AutoModelForCausalLM.from_pretrained("drive/My Drive/output_by_dialogue_pairs_re/")
# manga109の漫画ラブひなの役3000件のテキストデータでファインチューニングしたもの
# model = AutoModelForCausalLM.from_pretrained("drive/My Drive/output_by_LoveHina/")
# 上記のテキストデータのうち、発話者がnaruのものだけを抽出してファインチューニングしたもの
# model = AutoModelForCausalLM.from_pretrained("drive/My Drive/output_LoveHina_by_naru/")
# 無理やり2万件にしたもの
# model = AutoModelForCausalLM.from_pretrained("drive/My Drive/output_LoveHina_by_naru_inc_10000/")
# 無理やり1万件にしたもの
# model = AutoModelForCausalLM.from_pretrained("drive/My Drive/output_LoveHina_by_naru_inc_20000/")
# ラインのトーク履歴を使用
# model = AutoModelForCausalLM.from_pretrained("drive/My Drive/output_by_siraisi/")

In [ ]:
tokenizer.special_tokens_map

{'bos_token': '<s>',
 'eos_token': '</s>',
 'unk_token': '<unk>',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}

結果(ファインチューニング後)

In [ ]:
# 推論の実行
input = tokenizer.encode("こんちには", return_tensors="pt")
output = model.generate(input, do_sample=True, max_length=100, num_return_sequences=1)
# output = model.generate(input,do_sample=True,max_length=64, num_return_sequences=1,top_p=0.95, top_k=20, bad_words_ids=[[1], [5]], no_repeat_ngram_size=3)
print(tokenizer.batch_decode(output))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


['こんちには</s> <s> こんちには[SEP] 週末はどこか行きましたか?</s>']


結果(ファインチューニングなし)

In [2]:
from transformers import T5Tokenizer, AutoModelForCausalLM

# トークナイザーとモデルの準備
tokenizer_ = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-medium")
model_ = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt2-medium")

In [ ]:
# 推論の実行
input = tokenizer_.encode("こんばんわ", return_tensors="pt")
output = model_.generate(input, do_sample=True, max_length=100, num_return_sequences=8)
print(tokenizer_.batch_decode(output))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


['こんばんわ</s> 最近ご無沙汰しております・・・。この前、ふらっと街に出たのですが、なんだか風邪ひいたので帰ってきました というわけでブログのご報告遅れましたが、5月はライブが続き、今は新曲のレコーディングです!! そして5月にはライブとレコーディングが重なり、ちょっとブログの更新ができそうにないのでまたお邪魔しますね!ではでは</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>', 'こんばんわ</s> 。 昨日は、雨の中 沢山お越しいただき、ありがとうございまし～ん^^ とっても嬉しかったです ♪ おやすみなさいませー。 昨日のランチ 、今週もおいしゅうございました。 今夜は お肉を焼いたんですが、とっても甘くて おいしかったですー。 ランチ の後は デザートに 「メロンパン」(+600円', 'こんばんわ</s> ^^ 今日は、いつもより遅く起きて、2回目3回目に録画したものをまとめてみて、今日で終わりそう。 いや、でも、今日は、いつもより遅く起きるって決めたのは、昨晩の22時頃。 朝... 今日は、ちょっと前から欲しかったものを・・・・ 1ヶ月前にも買ったけど、まだ 1回ブランドもので、しかもちょっと高いし・・・・と思っていた、 財', 'こんばんわ</s> ☆ ☆ ☆ ☆ ☆ ☆ ☆ ☆ ☆ #meiko pic.twitter.com/1b3m0fzpxn @meikobefore_meiko @meiko_saki @meiko_stella @meiko_saki_michiruu2 @meiko_stella_michiruu2', 'こんばんわ</s> ブログを更新させていただいてから約一ヶ月。まだまだこれからの季節ですが、梅雨も明けて梅雨明けしましたね<unk> でも、雨の時期は一日中ずーっと雨が降っているので暑い日が続くのはやはり大変そうですね<unk> ところで、皆様、雨の日の外出に傘を忘れてしまったり、傘を収納スペースにいれたまま部屋を出てしまって傘を忘れてしまった・・・なんて', 'こんばんわ</s> 今日は朝からお出かけしてました。 お出かけはいつも行く場所で、 行ったところからの写真は 撮ってて楽しいのだけれど 撮った後、いつもと違う道を下る時 少しだけその道が違って見え

bilingual-gpt-neox-4b-instruction-sftを利用したファインチューニング(グラボのram不足のため実行不可)

In [ ]:
!python ./transformers/examples/pytorch/language-modeling/run_clm.py \
    --model_name_or_path=rinna/bilingual-gpt-neox-4b-instruction-sft \
    --train_file="drive/My Drive/dsbook/OpenNMT/siraisi_dialogue_re.txt" \
    --do_train \
    --do_eval \
    --num_train_epochs=10 \
    --save_steps=10000 \
    --block_size 512 \
    --save_total_limit=3 \
    --output_dir "drive/My Drive/output_by_siraisi/" \
    --overwrite_output_dir \
    --use_fast_tokenizer=False

結果

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("rinna/bilingual-gpt-neox-4b-instruction-sft", use_fast=False)
model = AutoModelForCausalLM.from_pretrained("rinna/bilingual-gpt-neox-4b-instruction-sft")

prompt = "こんにちは"

if torch.cuda.is_available():
    model = model.to("cuda")

token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        max_new_tokens=512,
        do_sample=True,
        temperature=1.0,
        top_p=0.85,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1):])
print(output)

OSError: ignored